# University Of Twente Databases

They have a few intersting tables.
1. All Databases
[UTwente All Satellites](https://webapps.itc.utwente.nl/sensor/default.aspx?view=allsatellites)

2. All Sensors Databases

Not sure how we will use the sensors database but might be useful, only scrapped the all satellites.

In [102]:
# Import necessary packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# Site URL
url="https://webapps.itc.utwente.nl/sensor/default.aspx?view=allsatellites"

In [103]:
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")

In [104]:
# On site there are 3 tables with the class "wikitable"
# The following line will generate a list of HTML content for each table
gdp = soup.find_all("table", attrs={"class": "sdbtable"})
print("Number of tables on site: ",len(gdp))

Number of tables on site:  1


In [105]:
# Lets go ahead and scrape first table with HTML code gdp[0]
table1 = gdp[0]
# the head will form our column names
body = table1.find_all("tr")
# Head values (Column names) are the first items of the body list
head = body[0] # 0th item is the header row
body_rows = body[1:] # All other items becomes the rest of the rows

# Lets now iterate through the head HTML code and make list of clean headings

# Declare empty list to keep Columns names
headings = []
for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    # append the clean column name to headings
    headings.append(item)

In [106]:
# Next is now to loop though the rest of the rows

#print(body_rows[0])
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

In [107]:
# We can now use the data on all_rowsa and headings to make a table
# all_rows becomes our data and headings the column names
unive_twente_satellites_df = pd.DataFrame(data=all_rows,columns=headings)
unive_twente_satellites_df.head()

Name           Orbit Type  \
0  \r                            ADEOS\r         ...      Sun Synchronous   
1  \r                            ADEOS-II\r      ...      Sun Synchronous   
2  \r                            Aeolus\r        ...      Sun Synchronous   
3  \r                            ALMAZ-1\r       ...  Non Sun Synchronous   
4  \r                            ALOS\r          ...      Sun Synchronous   

  Orbit Height (km) Repeat Cycle (days) Launched in Out of service since  \
0               797                  41        1996                 1997   
1               803                   4        2002                 2003   
2               320                   7        2018                        
3               370                            1991                 1992   
4               702                  46        2006                 2011   

              Organisation  
0             JAXA - Japan  
1             JAXA - Japan  
2                      ESA  
3  NPO - Mashinostroyeniya  
4             JAXA - Japan

In [108]:
unive_twente_satellites_df['Name'] = unive_twente_satellites_df['Name'].map(lambda x: x.strip())

In [109]:
unive_twente_satellites_df

Name           Orbit Type Orbit Height (km)  \
0               ADEOS      Sun Synchronous               797   
1            ADEOS-II      Sun Synchronous               803   
2              Aeolus      Sun Synchronous               320   
3             ALMAZ-1  Non Sun Synchronous               370   
4                ALOS      Sun Synchronous               702   
..                ...                  ...               ...   
329       WorldView-2      Sun Synchronous               770   
330       WorldView-3      Sun Synchronous               617   
331       WorldView-4      Sun Synchronous               617   
332  WorldView-Legion                Other                     
333        YaoGan - 5      Sun Synchronous                     

    Repeat Cycle (days) Launched in Out of service since  \
0                    41        1996                 1997   
1                     4        2002                 2003   
2                     7        2018                        
3                              1991                 1992   
4                    46        2006                 2011   
..                  ...         ...                  ...   
329                 1.1        2009                        
330                   1        2014                        
331                   1        2016                 2019   
332                            2021                        
333                            2008                        

                Organisation  
0               JAXA - Japan  
1               JAXA - Japan  
2                        ESA  
3    NPO - Mashinostroyeniya  
4               JAXA - Japan  
..                       ...  
329             DigitalGlobe  
330             DigitalGlobe  
331             DigitalGlobe  
332             DigitalGlobe  
333             CAST - China  

[334 rows x 7 columns]

In [110]:
unive_twente_satellites_df['Orbit Type'].unique()

array(['Sun Synchronous', 'Non Sun Synchronous', 'Geostationary',
       'near polar - sun synchronous', 'Other', 'Circular', 'Near polar'],
      dtype=object)

In [111]:
unive_twente_satellites_df.loc[unive_twente_satellites_df['Orbit Type'] == 'Circular']

Name Orbit Type Orbit Height (km) Repeat Cycle (days)  \
79              FORMOSAT-1   Circular               600                       
81     FORMOSAT-3 / COSMIC   Circular               800                       
83   FORMOSAT-7 / COSMIC-2   Circular               550                       
267                 SCISAT   Circular               650                       
293                  SORCE   Circular               645                   1   
305                   SWOT   Circular               891                  21   

    Launched in Out of service since                 Organisation  
79         1999                 2004                NSPO - Taiwan  
81         2006                 2020                NSPO - Taiwan  
83         2019                                     NSPO - Taiwan  
267        2003                       CSA - Canadian Space Agency  
293        2003                 2020                   NASA - USA  
305        2022                                        NASA - USA

In [112]:
unive_twente_satellites_df['Launched in'] = unive_twente_satellites_df['Launched in'].map(lambda x: int(x.strip()) if x != '' else x)

In [113]:
unive_twente_satellites_df['Launched in'].unique()

array([1996, 2002, 2018, 1991, 2006, 2014, 2021, 2022, 2016, 2010, 2004,
       2005, 2003, 2023, 2001, 2019, 2020, 2007, 2017, 2008, 1999, 2013,
       1987, 2009, 2015, 1994, 2011, 2000, '', 1995, 1997, 2024, 2012,
       1998, 1992, 1993, 1988, 2028, 2032, 1972, 1975, 1978, 1982, 1984,
       2031, 2036, 2025, 2039, 1990, 2042, 2034, 1986, 2026], dtype=object)

In [114]:
unive_twente_satellites_df.loc[unive_twente_satellites_df['Launched in'] == '']

Name     Orbit Type Orbit Height (km) Repeat Cycle (days)  \
62  EO-3 GIFTS  Geostationary             36000                       

   Launched in Out of service since Organisation  
62                                    NASA - USA

In [115]:
unive_twente_satellites_df.loc[62,'Launched in'] = 1

In [116]:
geostationary_df = unive_twente_satellites_df.loc[(unive_twente_satellites_df['Orbit Type'] == 'Geostationary') & 
                               (unive_twente_satellites_df['Launched in'] < 2000)].sort_values(by=['Launched in'])

In [117]:
geostationary_df

Name     Orbit Type Orbit Height (km) Repeat Cycle (days)  \
62         EO-3 GIFTS  Geostationary             36000                       
171        Meteosat 5  Geostationary             35786                       
127          INSAT-2A  Geostationary                                         
128          INSAT-2B  Geostationary                                         
172        Meteosat 6  Geostationary             35786                       
57   Electro-1 (GOMS)  Geostationary                                         
104            GOES-8  Geostationary             35800                       
94              GMS-5  Geostationary                                         
105            GOES-9  Geostationary             35800                       
70         Fengyun-2A  Geostationary                                         
96            GOES-10  Geostationary             35800                       
173        Meteosat 7  Geostationary             36000                0.01   
129          INSAT-2E  Geostationary                                         

    Launched in Out of service since          Organisation  
62            1                                 NASA - USA  
171        1991                 2007              EUMETSAT  
127        1992                 2002          ISRO - India  
128        1993                 2004          ISRO - India  
172        1993                 2011              EUMETSAT  
57         1994                 2000  ROSHYDROMET - Russia  
104        1994                 2004                  NOAA  
94         1995                 2003           JMA - Japan  
105        1995                 2007                  NOAA  
70         1997                 2000          NSMC - China  
96         1997                 2009                  NOAA  
173        1997                 2017              EUMETSAT  
129        1999                 2012          ISRO - India

In [120]:
unive_twente_satellites_df.loc[unive_twente_satellites_df['Name'] == 'Sentinel']

Empty DataFrame
Columns: [Name, Orbit Type, Orbit Height (km), Repeat Cycle (days), Launched in, Out of service since, Organisation]
Index: []

In [121]:
unive_twente_satellites_df.to_csv('./satellites.csv')